<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

##### MATHEMATICAL OPTIMIZATION
**Bachelor Degree in Data Science and Engineering** - Alex Carrillo Alza, Xavier Rubiés Cullell

$$
\LARGE \textsf{Lab assignment 2}
$$
$$
\huge \textsf{Ridge regression in Python}
$$

#### Assignment

- Implement and solve in Python the constrained ridge regression model:
$$
\begin{array}{rl}        \displaystyle   \min_{w,\gamma} & \frac{1}{2} (Aw+\gamma-y)^{\top} (Aw+\gamma-y)\\        \hbox{s. to} & \|w\|_2^2 \le t      \end{array}
$$


- You have to implement code for the evaluation of:
$$f(x), \nabla f(x),  \nabla^2 f(x), h_i(x), \nabla h_i(x),  \nabla^2 h_i(x), i=1,\dots,m$$

- Try your implementation with the attached `death_rate` dataset.


- Check that you obtain the same result as with the AMPL implementation  of ridge regression.


- The report must include all the previous elements (python code, results obtained, analysis of results, etc).


- This assignment is to be done in groups of two students.

### 1. Introduction

> The ridge or Tikhonov-regularized regression is based on finding an affine function, $y = w^{\top}x + \gamma, \quad w \in \mathbb R^n, \gamma \in \mathbb R$, which solves the **constrained problem**

$$
\begin{array}{rl}        \displaystyle   \min_{w,\gamma} & \frac{1}{2} (Aw+\gamma e-y)^{\top} (Aw+\gamma e-y)\\        \hbox{s. to} & \|w\|_2^2 \le t      \end{array}
$$

> An alternative (but different) *unconstrained* ridge regression model is (for some parameter $\mu$)

$$
\min_{w,\gamma} \frac{1}{2} (Aw+\gamma e-y)^{\top} (Aw+\gamma e-y) + \mu(\|w\|_2^2 - t)
$$

> Ridge regression is numerically better than standard regression (e.g., if $A^{\top}A$ is close to singular).

### 2. Preliminary calculations

> We are given $m$ points $(x_i,y_i), i = 1,\dots,m$, where $x_i \in \mathbb R^n, y \in \mathbb R$. Define matrix $A$ and vector $y$

$$
A = \begin{bmatrix} x_1^{\top} \\
                    \vdots \\
                    x_m^{\top}
\end{bmatrix}
\qquad
y = \begin{bmatrix} y_1 \\
                    \vdots \\
                    y_m
\end{bmatrix}
$$

> Considering $\mathbf{f(x)} := f(w, \gamma)$ and $\mathbf{h(x)} := h(w, \gamma)$ where $w \in \mathbb R^n, \gamma \in \mathbb R$, define

$$
w = \begin{bmatrix} w_1 & \cdots & w_n \end{bmatrix}^{\top}
\qquad
\gamma \in \mathbb R
\qquad
e = \begin{bmatrix} 1 & \overset{m}{\dotsb} & 1 \end{bmatrix}^{\top}
$$

> We get the following functions, gradients and hessians:

$$
\mathbf{f(x)} = \frac{1}{2} (Aw+\gamma e-y)^{\top} (Aw+\gamma e-y) \in \mathbb R
$$

$$
{\mathbf{\nabla f(x)} = \begin{bmatrix} \frac{\partial f}{\partial w} \\
\frac{\partial f}{\partial \gamma}
\end{bmatrix} =
\begin{bmatrix} A^{\top}(Aw + \gamma e - y) \\
e^{\top}Aw + \gamma m - e^{\top}y
\end{bmatrix}}_{(n+1) \times 1}
\qquad
{\mathbf{\nabla^2 f(x)} = \begin{bmatrix} \frac{\partial^2 f}{\partial w^2} & \frac{\partial^2 f}{\partial w \gamma} \\
\frac{\partial^2 f}{\partial \gamma w} & \frac{\partial^2 f}{\partial \gamma^2}
\end{bmatrix} =
\begin{bmatrix} A^{\top}A & A^{\top}e \\
e^{\top}A & m
\end{bmatrix}}_{(n+1) \times (n+1)}
$$

$$
\mathbf{h_i(x)} = \|w\|_2^2 = w^{\top}w \in \mathbb R
\qquad \qquad
{\mathbf{\nabla h_i(x)} = \begin{bmatrix} \frac{\partial h}{\partial w} \\
\frac{\partial h}{\partial \gamma}
\end{bmatrix} =
\begin{bmatrix} 2w \\
0
\end{bmatrix}}_{(n+1) \times 1}
$$

$$
{\mathbf{\nabla^2 h_i(x)} = \begin{bmatrix} \frac{\partial^2 h}{\partial w^2} & \frac{\partial^2 h}{\partial w \gamma} \\
\frac{\partial^2 h}{\partial \gamma w} & \frac{\partial^2 h}{\partial \gamma^2}
\end{bmatrix} =
\begin{bmatrix} 2 & 0 & \dotsb & 0 & 0 \\
0 & 2 & & & 0 \\
\vdots & & \ddots & & \vdots \\
0 & & & 2 & 0 \\
0 & 0 & \dotsb & 0 & 0 \\
\end{bmatrix}}_{(n+1) \times (n+1)}
$$

### 3. Implementation and test
 
> Implementation code and test with the `death_rate` dataset.

In [2]:
# Import libraries needed for optimization.
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

In [3]:
# Read dataset file.
data = np.loadtxt("death_rate.dat")

In [4]:
A = data[:,:-1]     # Matrix of explanatory variables.    m·n
y = data[:,-1]      # Vector of the response variable.    m·1

m = (A.shape)[0]    # Number of rows of matrix A.
n = (A.shape)[1]    # Number of columns of matrix A.

w = np.zeros(n)     # Vector of weights.                  n·1
gamma = 0           # Intercept parameter.                1·1
e = np.ones(m)      # Vector of ones.                     m·1

In [5]:
''' OBJECTIVE FUNCTION f(x) '''
def obje_f(x):
    # 'x' is a vector containing:    the vector of weights 'w'
    #                                the intercept parameter 'gamma'
    w = x[:-1]
    gamma = x[-1]
    
    # Returns the evaluation of the objective function f(x).    1·1
    form = np.matmul(A,w) + gamma*e - y    # Aw + ge - y
    return(1/2 * np.dot(form,form))

In [6]:
''' GRADIENT OF THE OBJECTIVE FUNCTION ∇f(x) '''
grad = np.zeros(n+1)    # Initialization of the gradient vector.

def grad_f(x):
    # 'x' is a vector containing:    the vector of weights 'w'
    #                                the intercept parameter 'gamma'
    w = x[:-1]
    gamma = x[-1]
    
    # Returns the evaluation of the gradient vector ∇f(x).    (n+1)·1
    grad[:-1] = np.matmul(np.transpose(A),(np.matmul(A,w) + gamma*e - y))           # A'(Aw + ge - y)
    grad[-1] = np.matmul(np.transpose(e),np.matmul(A,w)) + gamma*m - np.dot(e,y)    # e'Aw + gm - e'y
    return(grad)

In [7]:
''' HESSIAN OF THE OBJECTIVE FUNCTION ∇^(2)f(x) '''
hess = np.zeros((n+1,n+1))                    # Initialization of the hessian matrix.
hess[:n,:n] = np.matmul(np.transpose(A),A)    # Computation of the first 'n·n' inputs.    A'*A
hess[:n,-1] = np.matmul(np.transpose(A),e)    # Computation of the 'n·(n+1)' column.      A'*e
hess[-1,:n] = np.transpose(hess[:n,-1])       # Computation of the '(n+1)·n' row.         e'*A
hess[-1,-1] = m                               # Computation of the '(n+1)·(n+1)' element.   m

def hess_f(x):
    # Returns the evaluation of the hessian matrix ∇^(2)f(x).    (n+1)·(n+1)
    return(hess)

In [8]:
''' CONSTRAINT CONDITION h(x) '''
def cons_f(x):
    # 'x' is a vector containing:    the vector of weights 'w'
    #                                the intercept parameter 'gamma'
    w = x[:-1]
    
    # Returns the evaluation of the constraint condition h(x).    1·1
    return(np.dot(w,w))

In [9]:
''' JACOBIAN OF THE CONSTRAINT ∇h(x) '''
jaco = np.zeros(n+1)    # Initialization of the jacobian matrix.

def cons_J(x):
    # 'x' is a vector containing:    the vector of weights 'w'
    #                                the intercept parameter 'gamma'
    w = x[:-1]
    
    # Returns the evaluation of the jacobian matrix ∇h(x).    (n+1)·1
    jaco[:-1] = 2*w
    return(jaco)

In [10]:
''' HESSIAN OF THE CONSTRAINT ∇h^(2)(x) '''
hessC = 2*np.eye(n+1)
hessC[-1,-1] = 0         # Initialization of the hessian matrix.

def cons_H(x, v):
    # Returns the evaluation of the sum of Lagrange multipliers by the hessian matrix.    1·1
    return(v[0]*hessC)

In [11]:
# Definition of the nonlinear constraint (with the hiperparameter 't').
t = 1.0
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, t, jac = cons_J, hess = cons_H)

In [12]:
x0 = np.zeros(n+1)    # Definition of the starting point.

sol = minimize(obje_f, x0, method = 'trust-constr', jac = grad_f, hess = hess_f,    # Solve problem.
               constraints = [nonlinear_constraint], options = {'verbose': 1})

np.set_printoptions(suppress = True)
print(sol)            # Display of the obtained solution.

`xtol` termination condition is satisfied.
Number of iterations: 123, function evaluations: 182, CG iterations: 617, optimality: 1.92e-04, constraint violation: 0.00e+00, execution time: 0.22 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 617
      cg_stop_cond: 2
            constr: [array([1.])]
       constr_nfev: [182]
       constr_nhev: [65]
       constr_njev: [59]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.2233867645263672
               fun: 61484.654640764944
              grad: array([-10531.43189764,   1000.60086433,  -2221.25312436,    813.32934107,
         -105.85284749,    731.60168261,   5517.35190524,   -121.83830841,
       -14014.33632358,   2722.5970384 ,  -4603.2434718 ,   4482.96567861,
        -2369.32573194,  -8124.22726336,   -214.83474213,     -0.00000456])
               jac: [array([[ 0.97647914, -0.09277617,  0.20595561, -0.07541226,  0.00981472,
        -0.06783444, -0.

> With the implementation above we get a successful result. It takes only $123$ iterations and $182$ function evaluations to converge with a total execution time of **0.21 seconds**.

> Firstly, regarding optimization conditions it should be noted that:
1. There is no constraint violation
2. The values of the Lagrangian gradient are, in practice, zero

> In order to interpret the results better, we consider the sufficient optimality conditions:

> The point $x^*$ is local minimizer if: *First-order conditions (KKT)*
1. $h(x^*)=0$, $g(x^*) \leq 0$
2. $\nabla_x L(x^*,\lambda^*,\mu^*) = \nabla f(x*)+\nabla h(x^*)\lambda^*+\nabla g(x^*)\mu^*=0$
3. $\mu^* \geq 0$ and ${\mu^*}^T g(x^*)=0$ $\quad$ (if 
$g_j(x^*)$ is inactive then $\mu_j^*=0$)

> *Second-order conditions*
4. $d^T \nabla_{xx}^2 L(x^*,\lambda^*,\mu^*) d > 0$, for all $d \in M'=\{d: \nabla h(x^*)^T d = 0, \nabla g_j(x^*)^T d = 0, j\in A(x^*) \cap \{j: \mu_j^* > 0 \} \}$

> As ridge regression is a convex optimization problem, we only need to check the first-order conditions (KKT), because the property 4 is satisfied.

In [53]:
# Condition 1.
cons_f(sol.x) - t # <-- g(x*)

-3.009315019397718e-11

> - As we do not have $h(x^*)=0$ restriccion, we only have to evaluate $g(x^*) \leq 0$. Indeed, the condition is satisfied.

In [26]:
# Condition 2.
sol.lagrangian_grad # <-- ∇L(x*)

array([-0.00014326, -0.00003118,  0.0001211 ,  0.00012179, -0.0000665 ,
       -0.00001314, -0.00013913, -0.00000032, -0.00006218,  0.00005154,
        0.00015817, -0.00011087,  0.00019184, -0.00001733,  0.00004808,
       -0.00000456])

> - The values of the Lagrangian gradient are, in practice, zero. Thus, the condition is satisfied.

In [54]:
# Condition 3.
print(sol.v) # <-- mu
(sol.v[0][0])*(cons_f(sol.x)-1) # <-- mu·g(x*)

[array([10785.10672666])]


-3.2455783658349524e-07

> - The value of $\mu^*$ is greater than zero and the product of ${\mu^*}^T g(x^*)$ is, in practice, zero. Thus, the condition is satisfied.

> Taking all the above into account, we can conclude that our results are an optimal solution to the proposed problem.

### 4. Comparison with AMPL

> With the optimization software AMPL we get the following results for the `death_rate` dataset:

$$
\text{MINOS 5.51: optimal solution found.} \\
\text{133 iterations, objective 61484.65464} \\
\text{Nonlin evals: obj = 361, grad = 360, constrs = 361, Jac = 360.}
$$

$$
\text{w [*] :=} \\
1.\text{ 0.48824} \\
2.\text{ -0.0463881} \\
3.\text{ 0.102978} \\
4.\text{ -0.0377061} \\
5.\text{ 0.00490736} \\
6.\text{ -0.0339172} \\
7.\text{ -0.255786}\\ 
8.\text{ 0.00564845} \\
9.\text{ 0.649708} \\
10.\text{ -0.12622} \\
11.\text{ 0.213407} \\
12.\text{ -0.207831} \\ 
13.\text{ 0.109842} \\
14.\text{ 0.376641} \\
15.\text{ 0.00995978;} \\
\text{gamma = 895.141} \\
\text{norm2_w = -10785.1}
$$

> The first remark we observe is that we get the exact same results with AMPL for the optimal solution. Comparing the values of the objective function, the weights, the gamma and the norm, we observe no differences regarding our implementation.

> A curious fact is that, although AMPL performs a couple of iterations more than our implementation, it obtains a better time when it comes to finding the optimal solution (due to the internal resolution algorithms) with only *0.009984* seconds.